<a href="https://colab.research.google.com/github/treekeaw1/-mana-bento-web/blob/main/SPiAv36%E0%B8%84%E0%B8%B0%E0%B9%81%E0%B8%99%E0%B8%99%E0%B8%AA%E0%B8%B9%E0%B8%87%E0%B8%AA%E0%B8%B8%E0%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta

# --- เซลล์ที่ 1: การเตรียมและทำความสะอาดข้อมูล ---

print("--- เซลล์ที่ 1: การเตรียมและทำความสะอาดข้อมูล ---")
print("🔄 กำลังโหลดและทำความสะอาดข้อมูลจาก 'ไฟล์พร้อมใช้งาน.csv'...")

file_path = 'ไฟล์พร้อมใช้งาน.csv'
df = None
original_df_rows_count = 0 # จะนับจำนวนแถวในไฟล์ดิบก่อนการประมวลผล

try:
    # 1. โหลดไฟล์ CSV โดยข้ามบรรทัดแรก (metadata header)
    # ใช้บรรทัดที่สอง (ซึ่งตอนนี้เป็นบรรทัดแรกหลัง skip) เป็น Header จริงๆ
    # และกำหนด dtype ของคอลัมน์ตัวเลขให้เป็น string เพื่อป้องกัน .0
    encodings_to_try = ['utf-8', 'tis-620', 'cp874', 'latin1', 'iso-8859-1']

    # กำหนดชื่อคอลัมน์ที่เราคาดหวังจากภาพตัวอย่าง
    expected_header_names = ['วันที่', 'รางวัลที่ 1 (6 หลัก)', 'เลข 2 ตัวล่าง']

    for encoding in encodings_to_try:
        try:
            # skiprows=1: ข้ามบรรทัดแรกที่เป็น metadata
            # header=0: ใช้บรรทัดแรกหลัง skip (คือบรรทัดที่สองของไฟล์เดิม) เป็น Header
            # dtype: กำหนดให้คอลัมน์ที่คาดว่าจะเป็นตัวเลข ถูกอ่านเป็น string ทันที
            # (เราจะยังไม่รู้ชื่อคอลัมน์จริงที่ Pandas อ่านได้ในตอนนี้ แต่รู้ว่ามันคือคอลัมน์ที่ 1 และ 2)
            # เราจะกำหนด dtype เป็น 'object' (ซึ่งก็คือ string) สำหรับคอลัมน์ที่ 1 และ 2
            df_temp = pd.read_csv(file_path,
                                  encoding=encoding,
                                  skiprows=1,
                                  header=0,
                                  dtype={1: str, 2: str}) # กำหนดให้คอลัมน์ที่ 1 และ 2 เป็น string

            # ตรวจสอบว่ามีจำนวนคอลัมน์ที่เพียงพอและข้อมูลในแถวแรกไม่ว่างเปล่า
            if df_temp.shape[1] >= 3 and pd.notna(df_temp.iloc[0, 0]):
                df = df_temp
                print(f"✅ โหลดไฟล์ด้วย encoding '{encoding}' โดยข้าม 1 บรรทัดและใช้ Header สำเร็จ")
                break
            else:
                print(f"❌ โหลดไฟล์ด้วย encoding '{encoding}' สำเร็จ แต่ข้อมูลดูไม่ถูกต้อง (คอลัมน์ไม่พอ หรือแถวแรกว่าง)")

        except Exception as e:
            # print(f"❌ ไม่สามารถโหลดไฟล์ด้วย encoding '{encoding}': {e}") # ปิด log error ซ้ำๆ
            continue

    if df is None:
        raise Exception("ไม่สามารถโหลดไฟล์ CSV ด้วย encoding ใดๆ ที่ลองได้ หรือไฟล์ไม่มีข้อมูลที่ถูกต้อง")

    # 2. กำหนดชื่อคอลัมน์มาตรฐาน และตรวจสอบจำนวนคอลัมน์
    if len(df.columns) < 3:
        raise ValueError(f"จำนวนคอลัมน์ที่พบ ({len(df.columns)}) ไม่พอสำหรับ วันที่, รางวัลที่ 1, เลข 2 ตัวล่าง (ต้องมีอย่างน้อย 3 คอลัมน์)")

    # ใช้ df.columns[0], df.columns[1], df.columns[2] ซึ่งเป็นชื่อคอลัมน์ที่ Pandas อ่านได้ (แม้จะเป็นอักขระแปลกๆ)
    # แล้วเปลี่ยนชื่อคอลัมน์ให้เป็นชื่อมาตรฐานของเราทันที
    df_selected = df.iloc[:, [0, 1, 2]].copy() # เลือกแค่ 3 คอลัมน์แรก
    df_selected.columns = ['Date_Raw', 'Prize1_Raw', 'Last2Digits_Raw']

    print(f"\n✅ กำหนดให้คอลัมน์ '{df.columns[0]}' เป็น วันที่")
    print(f"✅ กำหนดให้คอลัมน์ '{df.columns[1]}' เป็น รางวัลที่ 1 (6 หลัก)")
    print(f"✅ กำหนดให้คอลัมน์ '{df.columns[2]}' เป็น เลข 2 ตัวล่าง")

    # 3. ลบบรรทัดสุดท้ายที่เป็น footer (ถ้ามี)
    last_row_date_val = str(df_selected.iloc[-1]['Date_Raw']).strip()
    if "artifact" in last_row_date_val.lower():
        print(f"ℹ️ ตรวจพบบรรทัดสุดท้าย ('{last_row_date_val[:50]}...') น่าจะเป็น Footer/Metadata จะลบออก")
        df_selected = df_selected.iloc[:-1].copy().reset_index(drop=True)

    original_df_rows_count = len(df_selected) # นับจำนวนแถวใน df_selected ก่อนทำความสะอาด

    # 4. ทำความสะอาดข้อมูลแต่ละแถว (ปรับปรุงการจัดการ NaN, '-' และเติม 0)
    cleaned_data = []
    seen_ids = set() # สำหรับกรองข้อมูลซ้ำ

    for index, row in df_selected.iterrows():
        try:
            # 4.1 แปลงวันที่ (รองรับหลายรูปแบบและจัดการค่าว่าง)
            date_raw = str(row['Date_Raw']).strip()
            if not date_raw or date_raw.lower() == 'nan' or date_raw == '-':
                continue

            date_obj = None
            date_formats = [
                '%Y/%m/%d', '%Y-%m-%d', '%d/%m/%Y', '%d-%m-%Y',
                '%d/%m/%y', '%d-%m-%y', # DD/MM/YY
                '%d %b %Y', '%d %B %Y', # DD MonYYYY (e.g., 1 ม.ค. 2565)
            ]

            # การแปลงปี พ.ศ. ให้เป็น ค.ศ. (ถ้าพบ)
            if re.search(r'\b25\d{2}\b', date_raw):
                date_raw_parts = date_raw.split()
                for i, part in enumerate(date_raw_parts):
                    if part.isdigit() and len(part) == 4 and part.startswith('25'):
                        date_raw_parts[i] = str(int(part) - 543)
                date_raw = ' '.join(date_raw_parts)

            for fmt in date_formats:
                try:
                    date_obj = datetime.strptime(date_raw, fmt)
                    break
                except ValueError:
                    continue

            if date_obj is None:
                print(f"⚠️ แถวที่ {index} (ข้อมูลดิบวันที่ '{row['Date_Raw']}'): รูปแบบวันที่ไม่ถูกต้องหรือไม่รองรับ ข้ามแถวนี้")
                continue

            # 4.2 ทำความสะอาด Prize1_Raw (6 หลัก) และเติม 0
            prize1_raw = str(row['Prize1_Raw']).strip()
            if not prize1_raw or prize1_raw.lower() == 'nan' or prize1_raw == '-':
                continue

            prize1_cleaned = re.sub(r'[^\d]', '', prize1_raw) # ลบอักขระที่ไม่ใช่ตัวเลข

            # *** จุดสำคัญ: เติม 0 ด้านหน้าให้ครบ 6 หลัก ***
            prize1_cleaned = prize1_cleaned.zfill(6)

            if len(prize1_cleaned) != 6:
                # ถ้าหลังจาก zfill แล้วยังไม่ 6 หลัก แสดงว่าข้อมูลผิดปกติมาก
                print(f"⚠️ แถวที่ {index} (Prize1_Raw '{row['Prize1_Raw']}'): ตัวเลขรางวัลที่ 1 ไม่ครบ 6 หลักหลังทำความสะอาดและเติม 0 ({len(prize1_cleaned)} หลัก) ข้ามแถวนี้")
                continue

            digits = [int(d) for d in prize1_cleaned]

            # 4.3 ทำความสะอาด Last2Digits_Raw (2 หลัก) และเติม 0
            last2_cleaned = None
            last2_raw = str(row['Last2Digits_Raw']).strip()

            if not last2_raw or last2_raw.lower() == 'nan' or last2_raw == '-':
                last2_cleaned = prize1_cleaned[-2:] # ใช้ 2 หลักสุดท้ายของ Prize1_Raw หากข้อมูลว่าง
            else:
                last2_cleaned = re.sub(r'[^\d]', '', last2_raw)

            # *** จุดสำคัญ: เติม 0 ด้านหน้าให้ครบ 2 หลัก ***
            last2_cleaned = last2_cleaned.zfill(2)

            if len(last2_cleaned) != 2:
                # ถ้าหลังจาก zfill แล้วยังไม่ 2 หลัก แสดงว่าข้อมูลผิดปกติมาก
                print(f"⚠️ แถวที่ {index} (Last2Digits_Raw '{row.get('Last2Digits_Raw', 'N/A')}'): ตัวเลข 2 ตัวล่างไม่ครบ 2 หลักหลังทำความสะอาดและเติม 0 ข้ามแถวนี้")
                continue

            # กรองข้อมูลซ้ำโดยใช้ วันที่ + Prize1 เป็น ID
            row_id = f"{date_obj.isoformat()}_{prize1_cleaned}"
            if row_id in seen_ids:
                print(f"⚠️ แถวที่ {index}: พบข้อมูลซ้ำ (วันที่: {date_obj.isoformat()}, Prize1: {prize1_cleaned}) ข้ามแถวนี้")
                continue
            seen_ids.add(row_id)

            cleaned_data.append({
                'id': row_id,
                'date': date_obj,
                'dateStr': date_obj.strftime('%Y-%m-%d'),
                'prize1': prize1_cleaned,
                'last2': last2_cleaned,
                'digits': digits,
                'sum': sum(digits),
                'evenCount': sum(1 for d in digits if d % 2 == 0),
                'oddCount': sum(1 for d in digits if d % 2 == 1),
                'last2Int': int(last2_cleaned)
            })

        except Exception as e:
            print(f"❌ เกิดข้อผิดพลาดที่ไม่คาดคิดในแถวที่ {index}: {e}. ข้ามแถวนี้")
            continue

    # 5. จัดเรียงข้อมูลตามวันที่
    processed_data = sorted(cleaned_data, key=lambda x: x['date'])

    if not processed_data:
        raise Exception("ไม่พบข้อมูลที่ถูกต้องหลังจากทำความสะอาด โปรดตรวจสอบรูปแบบข้อมูลในไฟล์ CSV ของคุณ")

    print(f"\n✅ การเตรียมและทำความสะอาดข้อมูลสำเร็จ: ประมวลผลได้ {len(processed_data)} รายการ (จาก {original_df_rows_count} รายการในไฟล์ที่เลือกมา)")
    print(f"ช่วงข้อมูล: ตั้งแต่ {processed_data[0]['dateStr']} ถึง {processed_data[-1]['dateStr']}")

    _processed_data_cell1 = processed_data
    _original_df_rows_cell1 = original_df_rows_count

except FileNotFoundError:
    print(f"❌ ข้อผิดพลาด: ไม่พบไฟล์ '{file_path}' โปรดตรวจสอบชื่อและที่ตั้งของไฟล์")
except ValueError as ve:
    print(f"❌ ข้อผิดพลาดข้อมูล: {ve}")
except Exception as e:
    print(f"❌ เกิดข้อผิดพลาดทั่วไป: {e}")



--- เซลล์ที่ 1: การเตรียมและทำความสะอาดข้อมูล ---
🔄 กำลังโหลดและทำความสะอาดข้อมูลจาก 'ไฟล์พร้อมใช้งาน.csv'...
✅ โหลดไฟล์ด้วย encoding 'utf-8' โดยข้าม 1 บรรทัดและใช้ Header สำเร็จ

✅ กำหนดให้คอลัมน์ '…' เป็น วันที่
✅ กำหนดให้คอลัมน์ '…' เป็น รางวัลที่ 1 (6 หลัก)
✅ กำหนดให้คอลัมน์ 'ñ…' เป็น เลข 2 ตัวล่าง
ℹ️ ตรวจพบบรรทัดสุดท้าย ('</artifact identifier="lottery_csv" type="text/pla...') น่าจะเป็น Footer/Metadata จะลบออก

✅ การเตรียมและทำความสะอาดข้อมูลสำเร็จ: ประมวลผลได้ 560 รายการ (จาก 560 รายการในไฟล์ที่เลือกมา)
ช่วงข้อมูล: ตั้งแต่ 2002-01-16 ถึง 2025-06-16


In [5]:
# --- ส่วนเพิ่มเติม: แสดงตัวอย่างข้อมูลที่ประมวลผลแล้ว ---
if '_processed_data_cell1' in locals() and _processed_data_cell1:
    print("\n--- ตัวอย่างข้อมูลที่ประมวลผลแล้ว (6 แถวแรก) ---")
    for i, item in enumerate(_processed_data_cell1[:6]):
        print(f"  {item['dateStr']} | Prize1: {item['prize1']} | Last2: {item['last2']}")

    if len(_processed_data_cell1) > 6:
        print("\n--- ตัวอย่างข้อมูลที่ประมวลผลแล้ว (6 แถวสุดท้าย) ---")
        for i, item in enumerate(_processed_data_cell1[-6:]):
            print(f"  {item['dateStr']} | Prize1: {item['prize1']} | Last2: {item['last2']}")
    else:
        print("\nℹ️ มีข้อมูลน้อยกว่า 6 แถว จึงแสดงแค่ทั้งหมดที่มีไปแล้ว")




--- ตัวอย่างข้อมูลที่ประมวลผลแล้ว (6 แถวแรก) ---
  2002-01-16 | Prize1: 709670 | Last2: 14
  2002-02-01 | Prize1: 633270 | Last2: 40
  2002-02-16 | Prize1: 330853 | Last2: 92
  2002-03-01 | Prize1: 805590 | Last2: 55
  2002-03-16 | Prize1: 659152 | Last2: 04
  2002-04-01 | Prize1: 990543 | Last2: 57

--- ตัวอย่างข้อมูลที่ประมวลผลแล้ว (6 แถวสุดท้าย) ---
  2025-04-01 | Prize1: 669687 | Last2: 36
  2025-04-16 | Prize1: 266227 | Last2: 85
  2025-05-02 | Prize1: 213388 | Last2: 06
  2025-05-16 | Prize1: 251309 | Last2: 87
  2025-06-01 | Prize1: 559352 | Last2: 20
  2025-06-16 | Prize1: 507392 | Last2: 06


In [6]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterGrid
from collections import defaultdict
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# สมมติว่า processed_data ได้ถูกเตรียมไว้จากเซลล์ก่อนหน้า
# ตัวอย่าง: processed_data = [...]

# หากคุณยังไม่มี processed_data, คุณสามารถสร้าง dummy data เพื่อทดสอบได้
# ตัวอย่างข้อมูล processed_data (ควรมาจากเซลล์แรกที่คุณมี)
# processed_data = [
#     {'id': 1, 'dateStr': '2020-01-01', 'prize1': '123456', 'last2': '78', 'date': datetime(2020, 1, 1), 'last2Int': 78, 'sum': sum(map(int, '123456')), 'evenCount': sum(1 for d in map(int, '123456') if d % 2 == 0), 'digits': [1,2,3,4,5,6]},
#     {'id': 2, 'dateStr': '2020-01-16', 'prize1': '654321', 'last2': '23', 'date': datetime(2020, 1, 16), 'last2Int': 23, 'sum': sum(map(int, '654321')), 'evenCount': sum(1 for d in map(int, '654321') if d % 2 == 0), 'digits': [6,5,4,3,2,1]},
#     {'id': 3, 'dateStr': '2020-02-01', 'prize1': '789012', 'last2': '45', 'date': datetime(2020, 2, 1), 'last2Int': 45, 'sum': sum(map(int, '789012')), 'evenCount': sum(1 for d in map(int, '789012') if d % 2 == 0), 'digits': [7,8,9,0,1,2]},
#     {'id': 4, 'dateStr': '2020-02-16', 'prize1': '102938', 'last2': '67', 'date': datetime(2020, 2, 16), 'last2Int': 67, 'sum': sum(map(int, '102938')), 'evenCount': sum(1 for d in map(int, '102938') if d % 2 == 0), 'digits': [1,0,2,9,3,8]},
#     # เพิ่มข้อมูลย้อนหลังให้มากพอสำหรับการทดสอบ
#     {'id': 5, 'dateStr': '2020-03-01', 'prize1': '456789', 'last2': '12', 'date': datetime(2020, 3, 1), 'last2Int': 12, 'sum': sum(map(int, '456789')), 'evenCount': sum(1 for d in map(int, '456789') if d % 2 == 0), 'digits': [4,5,6,7,8,9]},
#     {'id': 6, 'dateStr': '2020-03-16', 'prize1': '987654', 'last2': '34', 'date': datetime(2020, 3, 16), 'last2Int': 34, 'sum': sum(map(int, '987654')), 'evenCount': sum(1 for d in map(int, '987654') if d % 2 == 0), 'digits': [9,8,7,6,5,4]},
#     {'id': 7, 'dateStr': '2020-04-01', 'prize1': '111222', 'last2': '56', 'date': datetime(2020, 4, 1), 'last2Int': 56, 'sum': sum(map(int, '111222')), 'evenCount': sum(1 for d in map(int, '111222') if d % 2 == 0), 'digits': [1,1,1,2,2,2]},
#     {'id': 8, 'dateStr': '2020-04-16', 'prize1': '333444', 'last2': '78', 'date': datetime(2020, 4, 16), 'last2Int': 78, 'sum': sum(map(int, '333444')), 'evenCount': sum(1 for d in map(int, '333444') if d % 2 == 0), 'digits': [3,3,3,4,4,4]},
#     {'id': 9, 'dateStr': '2020-05-01', 'prize1': '555666', 'last2': '90', 'date': datetime(2020, 5, 1), 'last2Int': 90, 'sum': sum(map(int, '555666')), 'evenCount': sum(1 for d in map(int, '555666') if d % 2 == 0), 'digits': [5,5,5,6,6,6]},
#     {'id': 10, 'dateStr': '2020-05-16', 'prize1': '777888', 'last2': '01', 'date': datetime(2020, 5, 16), 'last2Int': 1, 'sum': sum(map(int, '777888')), 'evenCount': sum(1 for d in map(int, '777888') if d % 2 == 0), 'digits': [7,7,7,8,8,8]},
#     # เพิ่มข้อมูลเข้าไปอีกเพื่อให้เพียงพอสำหรับ start_train_size ของ backtesting (300 งวด)
#     # ... (ต้องมีข้อมูลจริงของคุณมากกว่านี้)
# ]

# หาก processed_data มาจากเซลล์ที่ 1 แล้ว ให้รันต่อได้เลย
if 'processed_data' not in locals() or not processed_data:
    print("❌ Error: 'processed_data' ไม่ถูกพบหรือไม่มีข้อมูล! โปรดตรวจสอบให้แน่ใจว่าเซลล์ก่อนหน้าได้รันและสร้าง processed_data แล้ว")
    processed_data = [] # กำหนดค่าว่างเพื่อป้องกัน error ในการรันต่อ

# --- 1. Feature Engineering (Extended) ---
print("⚙️ กำลังสร้าง Sequential Features...")

def extract_sequential_features(data, lookback_window=10, long_lookback_window=50):
    features_list = []

    # Global counters for overall frequency and last seen (across full history processed so far)
    # Initialize for 6 positions, 10 digits each (0-9)
    overall_digit_counts = {pos: defaultdict(int) for pos in range(6)}
    overall_digit_last_seen = {pos: {digit: -1 for digit in range(10)} for pos in range(6)} # Stores index of last seen

    for index, item in enumerate(data):
        features = {}

        # Ensure all expected fields exist, provide default empty/zero if not
        # This is crucial for consistency, especially for dummy future rows
        features['date'] = item.get('date')
        features['dateStr'] = item.get('dateStr', '')
        features['prize1'] = item.get('prize1', '')
        features['last2'] = item.get('last2', '')
        features['last2Int'] = item.get('last2Int', 0)
        features['sum'] = item.get('sum', 0)
        features['evenCount'] = item.get('evenCount', 0)
        features['digits'] = item.get('digits', [0,0,0,0,0,0]) # Default to all zeros for dummy

        # New: Add oddCount here (Crucial fix)
        if 'digits' in item and len(item['digits']) == 6:
            features['oddCount'] = sum(1 for digit in item['digits'] if digit % 2 != 0)
        else:
            features['oddCount'] = 0.0 # Ensure it's a float for consistency

        # History for lookback calculations
        history = data[max(0, index - lookback_window) : index]
        full_history_to_current = data[max(0, index - long_lookback_window) : index] # Use a longer window for overall trends

        # Initialize position-specific features to 0.0 for robustness
        for pos in range(6):
            features[f'pos{pos}_value'] = item['digits'][pos] if 'digits' in item and len(item['digits']) > pos else 0.0
            features[f'pos{pos}_parity'] = 1 if (item['digits'][pos] % 2 == 0) else 0 if 'digits' in item and len(item['digits']) > pos else 0.0
            features[f'pos{pos}_avgLast3'] = 0.0
            features[f'pos{pos}_stdLast3'] = 0.0
            features[f'pos{pos}_trend'] = 0.0
            features[f'pos{pos}_shiftFromPrev'] = 0.0
            features[f'pos{pos}_deltaFromAvg'] = 0.0
            features[f'pos{pos}_minInWindow'] = 0.0
            features[f'pos{pos}_maxInWindow'] = 0.0
            features[f'pos{pos}_overallFreq'] = 0.0
            features[f'pos{pos}_lastSeenGap'] = 0.0
            features[f'pos{pos}_sameDateAvg'] = 0.0
            features[f'pos{pos}_sameDateFreq'] = 0.0
            features[f'pos{pos}_sameDateLastValue'] = 0.0
            features[f'pos{pos}_sameDateTrend'] = 0.0

        # General features, initialize to 0.0 or appropriate defaults
        features['avgSum'] = 0.0
        features['avgEvenCount'] = 0.0
        features['avgOddCount'] = 0.0 # Initialize avgOddCount
        features['sum_last3_total'] = 0.0
        features['even_count_last3_total'] = 0.0
        features['odd_count_last3_total'] = 0.0 # Initialize odd_count_last3_total
        features['last2_lastValue'] = 0.0
        features['last2_frequency_in_history'] = 0.0
        features['last2_gap'] = 0.0
        features['last2_sameDateAvg'] = 0.0
        features['last2_sameDateFreq'] = 0.0
        features['last2_sameDateLastValue'] = 0.0
        features['last2_sameDateTrend'] = 0.0
        features['last2_sameDateGapYears'] = 0.0

        features['dayOfWeek'] = 0
        features['month'] = 0
        features['dayOfMonth'] = 0
        features['weekOfYear'] = 0
        features['year'] = 0
        features['isFirstHalfOfMonth'] = 0
        features['isMidMonth'] = 0
        features['isEndMonth'] = 0

        # --- Basic Positional Features (Existing & Improved) ---
        for pos in range(6):
            pos_history = [h['digits'][pos] for h in history if 'digits' in h and len(h['digits']) > pos]
            if pos_history:
                features[f'pos{pos}_avgLast3'] = np.mean(pos_history[-3:]) if len(pos_history) >= 3 else np.mean(pos_history)
                features[f'pos{pos}_stdLast3'] = np.std(pos_history[-3:]) if len(pos_history) >= 3 else 0.0

                # Trend (difference between current and previous in lookback window)
                if len(pos_history) >= 2:
                    features[f'pos{pos}_trend'] = pos_history[-1] - pos_history[-2]
                else:
                    features[f'pos{pos}_trend'] = 0.0

                # New: Positional Shift from previous draw
                if index > 0 and 'digits' in data[index-1] and len(data[index-1]['digits']) > pos and 'digits' in item and len(item['digits']) > pos:
                    features[f'pos{pos}_shiftFromPrev'] = item['digits'][pos] - data[index-1]['digits'][pos]
                else:
                    features[f'pos{pos}_shiftFromPrev'] = 0.0

                # New: Delta from Average in lookback window
                if 'digits' in item and len(item['digits']) > pos:
                    features[f'pos{pos}_deltaFromAvg'] = item['digits'][pos] - features[f'pos{pos}_avgLast3']
                else:
                    features[f'pos{pos}_deltaFromAvg'] = 0.0

                # New: Min/Max in lookback window
                features[f'pos{pos}_minInWindow'] = np.min(pos_history)
                features[f'pos{pos}_maxInWindow'] = np.max(pos_history)
            else:
                # Handle case where pos_history is empty
                features[f'pos{pos}_avgLast3'] = 0.0
                features[f'pos{pos}_stdLast3'] = 0.0
                features[f'pos{pos}_trend'] = 0.0
                features[f'pos{pos}_shiftFromPrev'] = 0.0
                features[f'pos{pos}_deltaFromAvg'] = 0.0
                features[f'pos{pos}_minInWindow'] = 0.0
                features[f'pos{pos}_maxInWindow'] = 0.0

        # --- Overall Sum/Even/Odd Counts Features (Existing & Improved) ---
        full_sums_history = [h['sum'] for h in history if 'sum' in h]
        full_even_counts_history = [h['evenCount'] for h in history if 'evenCount' in h]
        full_odd_counts_history = [h['oddCount'] for h in history if 'oddCount' in h] # New: Odd count history

        if full_sums_history: features['avgSum'] = np.mean(full_sums_history)
        if full_even_counts_history: features['avgEvenCount'] = np.mean(full_even_counts_history)
        if full_odd_counts_history: features['avgOddCount'] = np.mean(full_odd_counts_history) # New: Avg odd count

        features['sum_last3_total'] = np.sum(full_sums_history[-3:]) if len(full_sums_history) >= 3 else 0.0
        features['even_count_last3_total'] = np.sum(full_even_counts_history[-3:]) if len(full_even_counts_history) >= 3 else 0.0
        features['odd_count_last3_total'] = np.sum(full_odd_counts_history[-3:]) if len(full_odd_counts_history) >= 3 else 0.0 # New: Odd count last 3 total

        # --- Last2-specific Features (Existing) ---
        last2_history = [h['last2Int'] for h in history if 'last2Int' in h]
        if last2_history:
            features['last2_lastValue'] = last2_history[-1]
            if 'last2Int' in item:
                features['last2_frequency_in_history'] = last2_history.count(item['last2Int'])
            else:
                features['last2_frequency_in_history'] = 0.0
        else:
            features['last2_lastValue'] = 0.0
            features['last2_frequency_in_history'] = 0.0

        last_same_index = -1
        if 'last2Int' in item:
            for i_hist in range(index - 1, -1, -1):
                if 'last2Int' in data[i_hist] and data[i_hist]['last2Int'] == item['last2Int']:
                    last_same_index = i_hist
                    break
        features['last2_gap'] = index - last_same_index if last_same_index >= 0 else index + 1

        # --- Same-Date-in-History Features (Improved) ---
        if 'date' in item and isinstance(item['date'], datetime):
            same_date_history = [h for h in data[:index] if 'date' in h and isinstance(h['date'], datetime) and h['date'].month == item['date'].month and h['date'].day == item['date'].day]
            if same_date_history:
                for pos in range(6):
                    pos_same_date_digits = [h['digits'][pos] for h in same_date_history if 'digits' in h and len(h['digits']) > pos]
                    if pos_same_date_digits:
                        features[f'pos{pos}_sameDateAvg'] = np.mean(pos_same_date_digits)
                        if 'digits' in item and len(item['digits']) > pos:
                            features[f'pos{pos}_sameDateFreq'] = pos_same_date_digits.count(item['digits'][pos]) / len(pos_same_date_digits)
                        else: features[f'pos{pos}_sameDateFreq'] = 0.0
                        features[f'pos{pos}_sameDateLastValue'] = pos_same_date_digits[-1]
                        # New: Trend on same date (difference between last two same-date occurrences)
                        if len(pos_same_date_digits) >= 2:
                            features[f'pos{pos}_sameDateTrend'] = pos_same_date_digits[-1] - pos_same_date_digits[-2]
                        else:
                            features[f'pos{pos}_sameDateTrend'] = 0.0
                    else: # If no same_date_digits for this position
                        features[f'pos{pos}_sameDateAvg'] = 0.0
                        features[f'pos{pos}_sameDateFreq'] = 0.0
                        features[f'pos{pos}_sameDateLastValue'] = 0.0
                        features[f'pos{pos}_sameDateTrend'] = 0.0


                same_date_last2_int = [h['last2Int'] for h in same_date_history if 'last2Int' in h]
                if same_date_last2_int:
                    features['last2_sameDateAvg'] = np.mean(same_date_last2_int)
                    if 'last2Int' in item:
                        features['last2_sameDateFreq'] = same_date_last2_int.count(item['last2Int']) / len(same_date_last2_int)
                    else: features[f'pos{pos}_sameDateFreq'] = 0.0 # Changed from last2_sameDateFreq to pos{pos}_sameDateFreq
                    features['last2_sameDateLastValue'] = same_date_last2_int[-1]
                    # New: Trend on same date for last2
                    if len(same_date_last2_int) >= 2:
                        features['last2_sameDateTrend'] = same_date_last2_int[-1] - same_date_last2_int[-2]
                    else:
                        features['last2_sameDateTrend'] = 0.0
                else: # If no same_date_last2_int
                    features['last2_sameDateAvg'] = 0.0
                    features['last2_sameDateFreq'] = 0.0
                    features['last2_sameDateLastValue'] = 0.0
                    features['last2_sameDateTrend'] = 0.0

                last_same_date_year_appeared = -1
                if 'last2Int' in item:
                    for hist_item in reversed(same_date_history):
                        if 'last2Int' in hist_item and hist_item['last2Int'] == item['last2Int']:
                            if 'date' in hist_item and isinstance(hist_item['date'], datetime):
                                last_same_date_year_appeared = hist_item['date'].year
                                break

                if last_same_date_year_appeared != -1 and 'date' in item and isinstance(item['date'], datetime):
                    features['last2_sameDateGapYears'] = item['date'].year - last_same_date_year_appeared
                elif same_date_history and 'date' in same_date_history[0] and isinstance(same_date_history[0]['date'], datetime) and 'date' in item and isinstance(item['date'], datetime):
                    features['last2_sameDateGapYears'] = item['date'].year - same_date_history[0]['date'].year + 1 # Gap from earliest same-date entry
                else:
                    features['last2_sameDateGapYears'] = 0.0
            else: # If no same_date_history at all
                for pos in range(6):
                    features[f'pos{pos}_sameDateAvg'] = 0.0
                    features[f'pos{pos}_sameDateFreq'] = 0.0
                    features[f'pos{pos}_sameDateLastValue'] = 0.0
                    features[f'pos{pos}_sameDateTrend'] = 0.0
                features['last2_sameDateAvg'] = 0.0
                features['last2_sameDateFreq'] = 0.0
                features['last2_sameDateLastValue'] = 0.0
                features['last2_sameDateTrend'] = 0.0
                features['last2_sameDateGapYears'] = 0.0


        # --- Date-based Features (Expanded) ---
        if 'date' in item and isinstance(item['date'], datetime):
            features['dayOfWeek'] = item['date'].weekday() # 0=Monday, 6=Sunday
            features['month'] = item['date'].month
            features['dayOfMonth'] = item['date'].day
            features['weekOfYear'] = item['date'].isocalendar()[1]
            features['year'] = item['date'].year

            # New: Half of Month indicators
            features['isFirstHalfOfMonth'] = 1 if item['date'].day <= 15 else 0
            features['isMidMonth'] = 1 if item['date'].day == 16 else 0 # Explicitly for 16th
            features['isEndMonth'] = 1 if item['date'].day >= 28 else 0 # Roughly end of month

        # --- New Features based on 6 Techniques ---

        # 1. Frequency & Recency (Overall for each digit 0-9 for each position)
        # Update overall_digit_counts and overall_digit_last_seen with current item's digits
        if 'digits' in item and len(item['digits']) == 6:
            for pos, digit in enumerate(item['digits']):
                overall_digit_counts[pos][digit] += 1
                overall_digit_last_seen[pos][digit] = index

        # Calculate overall frequency and last seen gap for the *target* digits (if available)
        for pos in range(6):
            all_pos_digits_full_history = [h['digits'][pos] for h in data[:index] if 'digits' in h and len(h['digits']) > pos]
            if all_pos_digits_full_history:
                # If target digit exists in current item, calculate its frequency
                if 'digits' in item and len(item['digits']) > pos:
                    features[f'pos{pos}_overallFreq'] = all_pos_digits_full_history.count(item['digits'][pos]) / len(all_pos_digits_full_history)
                else:
                    features[f'pos{pos}_overallFreq'] = 0.0 # If not, it remains 0

            # Last Seen Gap (number of draws since this digit last appeared at this position)
            if 'digits' in item and len(item['digits']) > pos:
                target_digit = item['digits'][pos]
                last_seen_index = -1
                for i_back in range(index - 1, -1, -1):
                    if 'digits' in data[i_back] and len(data[i_back]['digits']) > pos and data[i_back]['digits'][pos] == target_digit:
                        last_seen_index = i_back
                        break
                features[f'pos{pos}_lastSeenGap'] = index - last_seen_index if last_seen_index >= 0 else index + 1
            else:
                features[f'pos{pos}_lastSeenGap'] = 0.0 # Default if no digits for current item

        # 2. Sum & Even/Odd Patterns (Already covered somewhat by sum & evenCount and their averages)
        # No new explicit features here, as sum/evenCount/oddCount are already used.

        # 3. Positional Relationships (Difference from adjacent digits - new)
        if 'digits' in item and len(item['digits']) == 6:
            for i in range(5):
                features[f'pos{i}_diff_pos{i+1}'] = item['digits'][i] - item['digits'][i+1]
            # New: Difference from the first digit
            features[f'pos1_diff_pos0'] = item['digits'][1] - item['digits'][0]
            features[f'pos2_diff_pos0'] = item['digits'][2] - item['digits'][0]
            features[f'pos3_diff_pos0'] = item['digits'][3] - item['digits'][0]
            features[f'pos4_diff_pos0'] = item['digits'][4] - item['digits'][0]
            features[f'pos5_diff_pos0'] = item['digits'][5] - item['digits'][0]
        else: # Initialize if digits are not available (e.g., dummy future row)
            for i in range(5):
                features[f'pos{i}_diff_pos{i+1}'] = 0.0
            features[f'pos1_diff_pos0'] = 0.0
            features[f'pos2_diff_pos0'] = 0.0
            features[f'pos3_diff_pos0'] = 0.0
            features[f'pos4_diff_pos0'] = 0.0
            features[f'pos5_diff_pos0'] = 0.0

        # 4. Cycle Theory / Repetition (Same-Date-in-History covers some of this)
        # Already have 'last2_sameDateGapYears'. No new explicit feature from this point.

        # 5. Trend Analysis (Already covered: pos_trend, sameDateTrend, avgLast3, shiftFromPrev, deltaFromAvg)
        # No new explicit features here.

        # 6. Date-Based Influence (Expanded dayOfWeek, month, dayOfMonth, weekOfYear)
        # Added 'isFirstHalfOfMonth', 'isMidMonth', 'isEndMonth'

        features['id'] = item.get('id', index)  # Ensure 'id' is always present
        features_list.append(features)
    return features_list

features_list_full = extract_sequential_features(processed_data)
print(f"✅ สร้าง Sequential Features สำเร็จ: {len(features_list_full)} รายการ")

# กรองข้อมูล Features ที่มี 'digits' ครบ 6 หลัก (ใช้สำหรับ Train/Test)
clean_features_df = pd.DataFrame([f for f in features_list_full if 'digits' in f and len(f['digits']) == 6])
if clean_features_df.empty:
    print("❌ ไม่มีข้อมูล Features ที่มีเลข 6 หลักที่ถูกต้อง ไม่สามารถดำเนินการต่อได้")
    _ml_analysis_data_cell2 = {} # Clear data if no valid features
else:
    # กำหนดคอลัมน์ Features ที่จะใช้
    columns_to_drop = ['date', 'dateStr', 'prize1', 'last2', 'last2Int', 'sum', 'evenCount', 'digits', 'id']
    feature_cols_for_prediction = [col for col in clean_features_df.columns if col not in columns_to_drop]

    # --- 2. การเตรียมข้อมูล ML และ Time-based Split ---
    def prepare_ml_data_for_split(features_df_subset, feature_columns, scaler=None, fit_scaler=True):
        df = features_df_subset.copy()
        valid_feature_columns = [col for col in feature_columns if col in df.columns]
        X_raw = df[valid_feature_columns].copy()
        y_targets = {}
        # Only extract y_targets if 'digits' column exists in the subset
        if 'digits' in df.columns:
            for pos in range(6): y_targets[pos] = df['digits'].apply(lambda x: x[pos])
        else:
            pass # y_targets will remain empty

        numerical_cols = X_raw.select_dtypes(include=np.number).columns.tolist()

        # Handle cases where numerical_cols might be empty or scaler is not needed
        if not numerical_cols:
            X_final = X_raw.copy() # No numerical columns to scale, return as is
            return X_final, y_targets, scaler, numerical_cols

        if fit_scaler:
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X_raw[numerical_cols])
        elif scaler: # Use existing scaler to transform
            X_scaled = scaler.transform(X_raw[numerical_cols])
        else: # No scaler provided and not fitting
            X_scaled = X_raw[numerical_cols].values

        # Reconstruct X_final with scaled numerical columns and original non-numerical columns
        X_final = pd.DataFrame(X_scaled, columns=numerical_cols, index=X_raw.index)
        # Add back any non-numerical columns if they exist and were dropped by select_dtypes
        for col in X_raw.columns:
            if col not in numerical_cols:
                X_final[col] = X_raw[col]

        return X_final, y_targets, scaler, numerical_cols

    # --- 3. Deep Learning (LSTM) Helper Functions ---
    n_timesteps_lstm = 3 # จำนวนงวดที่ใช้เป็น Lookback ใน LSTM (สามารถปรับได้)
    # เพิ่มการกำหนดค่า epochs และ batch_size สำหรับ LSTM
    lstm_epochs = 50 # จำนวนรอบการฝึก
    lstm_batch_size = 32 # ขนาดของ Batch

    def create_lstm_sequences(X_data, y_data_dict, n_timesteps):
        """
        แปลงข้อมูล X และ y ให้เป็นรูปแบบที่เหมาะกับ LSTM (samples, timesteps, features)
        y_data_dict สามารถเป็น dict ว่างเปล่าได้ หากต้องการแค่ X_sequences
        """
        X_sequences, y_sequences_dict = [], defaultdict(list)

        if X_data.empty or len(X_data) < n_timesteps:
            return np.array([]), {} # Return empty if not enough data

        X_np = X_data.values

        # For LSTM, we predict the target of the *last* timestep in the sequence
        # So, if sequence is [t-n+1, ..., t], we predict y at t.
        for i in range(len(X_np) - n_timesteps + 1):
            X_sequences.append(X_np[i : (i + n_timesteps), :])

            if y_data_dict:
                for pos in range(6):
                    # Ensure the target index is valid for y_data_dict
                    if pos in y_data_dict and (i + n_timesteps - 1) < len(y_data_dict[pos]):
                        y_sequences_dict[pos].append(y_data_dict[pos].iloc[i + n_timesteps - 1])

        X_sequences = np.array(X_sequences)
        y_sequences_final_dict = {pos: np.array(y_list) for pos, y_list in y_sequences_dict.items() if y_list} if y_data_dict else {}

        return X_sequences, y_sequences_final_dict

    def create_lstm_model(input_shape):
        model = Sequential([
            LSTM(units=50, activation='relu', input_shape=input_shape, return_sequences=False),
            Dropout(0.2),
            Dense(units=10, activation='softmax') # 10 possible digits (0-9)
        ])
        model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        return model

    # --- 4. Hyperparameter Tuning อัตโนมัติ (สำหรับ RandomForest) ---
    print("\n🔬 กำลังดำเนินการ Hyperparameter Tuning สำหรับเลข 6 หลัก (RandomForest)...")
    param_grid_rf = {
        'n_estimators': [50, 100, 150, 200],
        'max_depth': [10, 20, None]
    }
    best_accuracy_rf = -1
    best_params_rf = {}
    best_position_accuracies_rf = {}

    # Use a fixed test set for tuning to compare hyperparameters fairly
    tuning_split_ratio = 0.8 # 80% for training, 20% for tuning validation
    tuning_train_end_index = int(len(clean_features_df) * tuning_split_ratio)

    # Ensure that tuning_X_raw_full includes 'digits' for target extraction during prepare_ml_data_for_split
    tuning_X_raw_full, tuning_y_targets_full, tuning_scaler, tuning_numerical_cols = prepare_ml_data_for_split(
        clean_features_df, feature_cols_for_prediction, fit_scaler=True
    )

    tuning_X_train = tuning_X_raw_full.iloc[:tuning_train_end_index]
    tuning_y_train = {pos: tuning_y_targets_full[pos].iloc[:tuning_train_end_index] for pos in range(6)}

    tuning_X_test = tuning_X_raw_full.iloc[tuning_train_end_index:]
    tuning_y_test = {pos: tuning_y_targets_full[pos].iloc[tuning_train_end_index:] for pos in range(6)}

    # Check if tuning_X_train or tuning_X_test are empty
    if tuning_X_train.empty or tuning_X_test.empty:
        print("❌ ข้อมูลไม่เพียงพอสำหรับการทำ Hyperparameter Tuning. โปรดตรวจสอบชุดข้อมูล.")
        best_params_rf = {'n_estimators': 100, 'max_depth': None} # Fallback to default
    else:
        for params in ParameterGrid(param_grid_rf):
            print(f"  > กำลังทดสอบ n_estimators={params['n_estimators']}, max_depth={params['max_depth']}...")
            current_position_accuracies = {}
            for pos in range(6):
                model = RandomForestClassifier(random_state=42, **params)
                model.fit(tuning_X_train, tuning_y_train[pos])
                accuracy = model.score(tuning_X_test, tuning_y_test[pos])
                current_position_accuracies[pos] = accuracy

            # Criterion for best parameters: Average accuracy of last 3 positions (4, 5, 6)
            avg_accuracy_last3 = np.mean([current_position_accuracies[pos] for pos in [3, 4, 5]]) # positions 4, 5, 6 are indices 3, 4, 5

            if avg_accuracy_last3 > best_accuracy_rf:
                best_accuracy_rf = avg_accuracy_last3
                best_params_rf = params
                best_position_accuracies_rf = current_position_accuracies.copy()

        print("✅ การทำ Hyperparameter Tuning เสร็จสิ้น")
        print("\n--- สรุปผลการ Tuning (RandomForest) ---")
        print(f"🥇 ค่า Hyperparameters ที่ดีที่สุด (อิงจากเฉลี่ยความแม่นยำตำแหน่ง 4, 5, 6):")
        print(f"  n_estimators: {best_params_rf.get('n_estimators')}")
        print(f"  max_depth: {best_params_rf.get('max_depth')}")
        print(f"  ค่าเฉลี่ยความแม่นยำ (ตำแหน่ง 4, 5, 6): {best_accuracy_rf:.4f}")
        print("  ความแม่นยำในแต่ละตำแหน่งด้วยค่าที่ดีที่สุด:")
        for pos, acc in best_position_accuracies_rf.items():
            print(f"    ตำแหน่งที่ {pos+1}: {acc:.4f}")

        # Calculate overall accuracy for 6 digits (all must be correct)
        correct_predictions_count = 0
        total_samples = len(tuning_X_test)
        if total_samples > 0:
            predictions_per_position = {}
            for pos in range(6):
                model = RandomForestClassifier(random_state=42, **best_params_rf)
                model.fit(tuning_X_train, tuning_y_train[pos])
                predictions_per_position[pos] = model.predict(tuning_X_test)

            for i in range(total_samples):
                all_correct_for_sample = True
                for pos in range(6):
                    if predictions_per_position[pos][i] != tuning_y_test[pos].iloc[i]:
                        all_correct_for_sample = False
                        break
                if all_correct_for_sample:
                    correct_predictions_count += 1
            overall_accuracy_rf = correct_predictions_count / total_samples
        else:
            overall_accuracy_rf = 0.0
        print(f"  ความแม่นยำรวม 6 หลักด้วยค่าที่ดีที่สุด: {overall_accuracy_rf:.4f}")


    # --- 5. Backtesting (Walk-Forward Validation) สำหรับเลข 6 หลัก (รองรับ RandomForest และ LSTM) ---
    def run_backtesting(model_type, features_df_full, feature_cols,
                            start_train_size=300, validation_size=10, step_size=10,
                            rf_params=None, lstm_timesteps=3, lstm_epochs=50, lstm_batch_size=32):

        print(f"\n📈 กำลังดำเนินการ Backtesting (Walk-Forward Validation) สำหรับเลข 6 หลัก ({model_type})...")

        all_position_accuracies = defaultdict(list)
        all_overall_accuracies = []

        # Ensure features_df_full is sorted by date/index
        features_df_full = features_df_full.sort_index()

        num_records = len(features_df_full)
        current_train_start = 0
        current_train_end = start_train_size

        fold_count = 0
        while current_train_end + validation_size <= num_records:
            fold_count += 1

            # Define train and validation sets for the current fold
            train_data_fold = features_df_full.iloc[current_train_start:current_train_end]
            validation_data_fold = features_df_full.iloc[current_train_end : current_train_end + validation_size]

            print(f"  > Fold {fold_count}: Training on {len(train_data_fold)} records, Validating on {len(validation_data_fold)} records...")

            # Prepare data for ML (scaling and target extraction)
            # Fit scaler on training data of the current fold
            X_train_raw, y_train_targets, scaler_fold, numerical_cols_fold = prepare_ml_data_for_split(
                train_data_fold, feature_cols, fit_scaler=True
            )
            # Transform validation data using the same scaler
            X_val_raw, y_val_targets, _, _ = prepare_ml_data_for_split(
                validation_data_fold, feature_cols, scaler=scaler_fold, fit_scaler=False
            )

            if X_train_raw.empty or X_val_raw.empty:
                print(f"    ⚠️ Fold {fold_count} ข้อมูลไม่เพียงพอ ข้าม Fold นี้")
                current_train_end += step_size
                current_train_start += step_size
                continue

            current_fold_position_accuracies = {}
            current_fold_predictions_per_position = {}

            # --- Model Training and Prediction based on model_type ---
            if model_type == 'random_forest':
                for pos in range(6):
                    model = RandomForestClassifier(random_state=42, **rf_params)
                    model.fit(X_train_raw, y_train_targets[pos])
                    accuracy = model.score(X_val_raw, y_val_targets[pos])
                    current_fold_position_accuracies[pos] = accuracy
                    current_fold_predictions_per_position[pos] = model.predict(X_val_raw)

            elif model_type == 'lstm':
                # Create LSTM sequences for training and validation
                X_train_lstm, y_train_lstm_dict = create_lstm_sequences(X_train_raw, y_train_targets, lstm_timesteps)
                X_val_lstm, y_val_lstm_dict = create_lstm_sequences(X_val_raw, y_val_targets, lstm_timesteps)

                if X_train_lstm.shape[0] == 0 or X_val_lstm.shape[0] == 0:
                    print(f"    ⚠️ Fold {fold_count} ข้อมูล LSTM Sequence ไม่เพียงพอ ข้าม Fold นี้")
                    current_train_end += step_size
                    current_train_start += step_size
                    continue

                # Check if y_train_lstm_dict and y_val_lstm_dict are populated for all positions
                # This ensures target data exists for training and validation
                if not y_train_lstm_dict or any(y_train_lstm_dict[pos].shape[0] == 0 for pos in range(6) if pos in y_train_lstm_dict) or \
                   not y_val_lstm_dict or any(y_val_lstm_dict[pos].shape[0] == 0 for pos in range(6) if pos in y_val_lstm_dict):
                        print(f"    ⚠️ Fold {fold_count} ข้อมูล Target สำหรับ LSTM ไม่ครบถ้วน ข้าม Fold นี้")
                        current_train_end += step_size
                        current_train_start += step_size
                        continue


                for pos in range(6):
                    # Check explicitly if target for current pos exists and is not empty
                    if pos not in y_train_lstm_dict or y_train_lstm_dict[pos].shape[0] == 0 or \
                       pos not in y_val_lstm_dict or y_val_lstm_dict[pos].shape[0] == 0:
                        print(f"    ⚠️ Fold {fold_count} ไม่มีข้อมูล Target ที่เพียงพอสำหรับตำแหน่งที่ {pos+1} ใน LSTM. ข้ามการฝึก LSTM.")
                        current_fold_position_accuracies[pos] = 0.0 # Default accuracy
                        current_fold_predictions_per_position[pos] = np.array([]) # Empty predictions
                        continue

                    lstm_model = create_lstm_model(input_shape=(lstm_timesteps, X_train_lstm.shape[2]))
                    lstm_model.fit(
                        X_train_lstm, y_train_lstm_dict[pos],
                        epochs=lstm_epochs,
                        batch_size=lstm_batch_size,
                        verbose=0 # Suppress output during training
                    )
                    loss, accuracy = lstm_model.evaluate(X_val_lstm, y_val_lstm_dict[pos], verbose=0)
                    current_fold_position_accuracies[pos] = accuracy
                    current_fold_predictions_per_position[pos] = np.argmax(lstm_model.predict(X_val_lstm), axis=1)

            # Store position accuracies for this fold
            for pos, acc in current_fold_position_accuracies.items():
                all_position_accuracies[pos].append(acc)

            # Calculate overall 6-digit accuracy for this fold
            correct_predictions_count_fold = 0
            total_samples_fold = X_val_lstm.shape[0] if model_type == 'lstm' else len(validation_data_fold)

            if total_samples_fold > 0:
                for i in range(total_samples_fold):
                    all_correct_for_sample = True
                    for pos in range(6):
                        # Corrected target lookup for LSTM validation
                        if model_type == 'lstm':
                            # Ensure index 'i' exists in y_val_lstm_dict[pos]
                            if pos not in y_val_lstm_dict or i >= len(y_val_lstm_dict[pos]):
                                all_correct_for_sample = False
                                break
                            actual_digit = y_val_lstm_dict[pos][i]
                        else: # RandomForest
                            actual_digit = y_val_targets[pos].iloc[i]


                        if pos not in current_fold_predictions_per_position or \
                           i >= len(current_fold_predictions_per_position[pos]):
                            all_correct_for_sample = False
                            break

                        predicted_digit = current_fold_predictions_per_position[pos][i]

                        if predicted_digit != actual_digit:
                            all_correct_for_sample = False
                            break
                    if all_correct_for_sample:
                        correct_predictions_count_fold += 1
                overall_accuracy_fold = correct_predictions_count_fold / total_samples_fold
            else:
                overall_accuracy_fold = 0.0
            all_overall_accuracies.append(overall_accuracy_fold)

            # Move to the next fold
            current_train_end += step_size
            current_train_start += step_size # Slide window

        print(f"✅ การทำ Backtesting เสร็จสิ้น ({fold_count} Folds)")

        print("\n--- ผลลัพธ์ Backtesting เฉลี่ย ---")
        avg_position_accuracies = {pos: np.mean(acc_list) for pos, acc_list in all_position_accuracies.items() if acc_list}
        for pos in range(6):
            print(f"  ความแม่นยำเฉลี่ยในแต่ละตำแหน่ง (ตำแหน่งที่ {pos+1}): {avg_position_accuracies.get(pos, 0.0):.4f}")

        avg_overall_accuracy = np.mean(all_overall_accuracies) if all_overall_accuracies else 0.0
        print(f"  ความแม่นยำรวม 6 หลักเฉลี่ย (ต้องถูกทุกหลัก): {avg_overall_accuracy:.4f}")

        return avg_position_accuracies, avg_overall_accuracy

    # เรียกใช้ Backtesting สำหรับ RandomForest (เฉพาะเมื่อมีข้อมูลเพียงพอสำหรับการจูน)
    if not best_params_rf: # If tuning failed or no data for tuning
        print("\n⚠️ ไม่มี best_params_rf ที่คำนวณได้จากการ Tuning, ใช้ค่า default สำหรับ Backtesting RandomForest.")
        rf_params_for_backtest = {'n_estimators': 100, 'max_depth': None}
    else:
        rf_params_for_backtest = best_params_rf

    rf_avg_pos_acc, rf_avg_overall_acc = run_backtesting(
        model_type='random_forest',
        features_df_full=clean_features_df,
        feature_cols=feature_cols_for_prediction,
        rf_params=rf_params_for_backtest
    )

    # เรียกใช้ Backtesting สำหรับ LSTM
    lstm_avg_pos_acc, lstm_avg_overall_acc = run_backtesting(
        model_type='lstm',
        features_df_full=clean_features_df,
        feature_cols=feature_cols_for_prediction,
        lstm_timesteps=n_timesteps_lstm, # ใช้ค่า n_timesteps_lstm ที่กำหนดไว้ด้านบน
        lstm_epochs=lstm_epochs, # ส่งค่าเข้าไปในฟังก์ชัน
        lstm_batch_size=lstm_batch_size # ส่งค่าเข้าไปในฟังก์ชัน
    )

    # --- 6. การทำนายเลข 6 หลัก (สำหรับงวดถัดไป) ---
    print("\n--- 🔢 กำลังทำนายเลข 6 หลัก (สำหรับงวดถัดไป) ---")

    # สร้างข้อมูล Features สำหรับงวดถัดไปโดยใช้ข้อมูลถึงงวดล่าสุด
    # เราต้องสร้างข้อมูลเสมือนของงวดถัดไปเพื่อคำนวณ Features

    if not processed_data:
        print("❌ ไม่พบข้อมูลใน processed_data ไม่สามารถทำนายงวดถัดไปได้")
        next_draw_date = datetime.now() # Fallback date
        rf_predicted_digits = ['N/A'] * 6
        rf_confidence_per_pos = [0.0] * 6
        rf_position_probabilities = {pos: [] for pos in range(6)}
        lstm_predicted_digits = ['N/A'] * 6
        lstm_confidence_per_pos = [0.0] * 6
        lstm_position_probabilities = {pos: [] for pos in range(6)}
        predicted_last2 = 'N/A'
        last2_confidence = 0.0
        last2_probabilities_top5 = []

    else:
        # ค้นหาวันที่งวดถัดไปที่ถูกต้อง (วันที่ 1 หรือ 16 ของเดือน)
        last_date = processed_data[-1]['date']
        if last_date.day == 1:
            next_draw_date = last_date.replace(day=16)
        elif last_date.day == 16:
            if last_date.month == 12:
                next_draw_date = last_date.replace(year=last_date.year + 1, month=1, day=1)
            else:
                next_draw_date = last_date.replace(month=last_date.month + 1, day=1)
        else: # กรณีที่ไม่ใช่วันที่ 1 หรือ 16 (ควรจัดการด้วยความระมัดระวัง)
            # หา 1 หรือ 16 ที่ใกล้ที่สุดในอนาคต
            temp_date = last_date + timedelta(days=1)
            while True:
                if temp_date.day == 1 or temp_date.day == 16:
                    next_draw_date = temp_date
                    break
                temp_date += timedelta(days=1)


        # สร้าง dummy data สำหรับงวดถัดไป (ไม่มีเลขรางวัลจริง)
        # Penting: 'digits', 'sum', 'evenCount', 'oddCount', 'last2Int' need to be present even as dummies
        # so that extract_sequential_features can process them without errors and include these feature columns.
        next_draw_dummy_data = {
            'date': next_draw_date,
            'dateStr': next_draw_date.strftime('%Y-%m-%d'),
            'prize1': '', 'last2': '', 'last2Int': 0,
            'sum': 0, 'evenCount': 0, 'oddCount': 0, # Explicitly include oddCount
            'digits': [0,0,0,0,0,0] # Dummy digits for feature calculation, won't affect prediction target
        }

        # สร้าง features_list_for_prediction โดยรวม dummy data เข้าไปด้วย
        processed_data_with_dummy = processed_data + [next_draw_dummy_data]
        features_list_for_prediction_context = extract_sequential_features(processed_data_with_dummy)

        # ดึง features ของงวด dummy (งวดสุดท้ายใน features_list_for_prediction_context)
        future_features_row_data = features_list_for_prediction_context[-1]
        future_features_df = pd.DataFrame([future_features_row_data])

        # เตรียม X สำหรับการทำนาย (ใช้ scaler ที่ fit จากข้อมูลทั้งหมดที่มี target)
        # ตรวจสอบว่า feature_cols_for_prediction มีคอลัมน์ที่เป็นตัวเลขจริงหรือไม่
        # เพื่อป้องกันข้อผิดพลาดเมื่อไม่มี numerical_cols
        X_full_scaled, y_full_targets, scaler_for_prediction, numerical_cols_for_scaling = prepare_ml_data_for_split(
            clean_features_df, feature_cols_for_prediction, fit_scaler=True # clean_features_df มี digits และเป็นข้อมูลสำหรับฝึก
        )

        # X สำหรับทำนายงวดถัดไป ต้องใช้ scaler_for_prediction มาแปลง
        X_predict_rf, _, _, _ = prepare_ml_data_for_split(
            future_features_df, feature_cols_for_prediction, scaler=scaler_for_prediction, fit_scaler=False
        )

        # ทำนายด้วย RandomForest (ใช้โมเดลที่ฝึกด้วย best_params_rf)
        rf_predicted_digits = []
        rf_confidence_per_pos = []
        rf_position_probabilities = {}

        for pos in range(6):
            model = RandomForestClassifier(random_state=42, **best_params_rf)
            # Train model on all available data for final prediction
            # Ensure X_full_scaled is used for training, as it's the scaled version of clean_features_df
            # y_full_targets contains the actual digits
            model.fit(X_full_scaled, y_full_targets[pos])

            # Ensure X_predict_rf is aligned and has the same columns as X_full_scaled
            # This should be handled by prepare_ml_data_for_split
            proba = model.predict_proba(X_predict_rf)[0]
            predicted_digit = np.argmax(proba)
            confidence = proba[predicted_digit] * 100 # Convert to percentage

            rf_predicted_digits.append(predicted_digit)
            rf_confidence_per_pos.append(confidence)
            rf_position_probabilities[pos] = sorted(zip(range(10), proba), key=lambda x: x[1], reverse=True)

        rf_overall_confidence = np.mean(rf_confidence_per_pos) # Simple average for now

        print(f"ชุดทำนายที่ 1 (RandomForest - ทำนายงวดถัดไป): **{''.join(map(str, rf_predicted_digits))}**")
        print(f"  ความเชื่อมั่นรวม: {rf_overall_confidence:.1f}%")
        print("  รายละเอียดแต่ละหลัก:")
        for pos, digit in enumerate(rf_predicted_digits):
            print(f"    ตำแหน่ง {pos+1}: เลข {digit} (ความเชื่อมั่น: {rf_confidence_per_pos[pos]:.0f}%)")

        # ทำนายด้วย LSTM (ใช้โมเดลที่ฝึกด้วย n_timesteps_lstm)
        lstm_predicted_digits = []
        lstm_confidence_per_pos = []
        lstm_position_probabilities = {}

        # ต้องสร้าง X_predict_lstm จากลำดับข้อมูลล่าสุด n_timesteps_lstm งวดจาก X_full_scaled
        if len(X_full_scaled) >= n_timesteps_lstm:
            # ดึง n_timesteps_lstm งวดย้อนหลังจาก X_full_scaled (ซึ่งเป็นข้อมูลที่สเกลแล้ว)
            X_predict_lstm_sequence_input = X_full_scaled.iloc[-(n_timesteps_lstm):].values.reshape(1, n_timesteps_lstm, X_full_scaled.shape[1])

            # create_lstm_sequences จะถูกเรียกในโหมดทำนาย (y_data_dict เป็น {} หรือไม่มีข้อมูล target)
            # แต่เราต้องการแค่ input สำหรับ predict, ไม่ได้สร้าง sequence สำหรับการเทรน
            # ดังนั้นจึงต้อง reshape ด้วยตนเองสำหรับ single prediction

            if X_predict_lstm_sequence_input.shape[0] > 0: # Check if sequence was successfully created
                for pos in range(6):
                    # Train LSTM model on all available data for final prediction
                    X_all_lstm, y_all_lstm_dict = create_lstm_sequences(
                        X_full_scaled, # ใช้ X_full_scaled เพราะเป็นข้อมูล Features ที่สเกลแล้ว
                        {p: clean_features_df['digits'].apply(lambda x: x[p]) for p in range(6)}, # target y จาก clean_features_df
                        n_timesteps_lstm
                    )
                    if X_all_lstm.shape[0] == 0:
                        print(f"    ⚠️ ไม่มีข้อมูล LSTM เพียงพอสำหรับฝึกโมเดลตำแหน่งที่ {pos+1} เพื่อทำนายงวดถัดไป")
                        lstm_predicted_digits.append('N/A')
                        lstm_confidence_per_pos.append(0.0)
                        lstm_position_probabilities[pos] = []
                        continue

                    lstm_model_final = create_lstm_model(input_shape=(n_timesteps_lstm, X_all_lstm.shape[2]))
                    lstm_model_final.fit(
                        X_all_lstm, y_all_lstm_dict[pos],
                        epochs=lstm_epochs, # **แก้ไข: เพิ่มตัวแปร lstm_epochs**
                        batch_size=lstm_batch_size, # **แก้ไข: เพิ่มตัวแปร lstm_batch_size**
                        verbose=0
                    )

                    proba = lstm_model_final.predict(X_predict_lstm_sequence_input)[0]
                    predicted_digit = np.argmax(proba)
                    confidence = proba[predicted_digit] * 100

                    lstm_predicted_digits.append(predicted_digit)
                    lstm_confidence_per_pos.append(confidence)
                    lstm_position_probabilities[pos] = sorted(zip(range(10), proba), key=lambda x: x[1], reverse=True)
            else:
                print("⚠️ ไม่สามารถสร้าง LSTM sequence สำหรับการทำนายงวดถัดไปได้ (ข้อมูลไม่เพียงพอ).")
                lstm_predicted_digits = ['N/A'] * 6
                lstm_confidence_per_pos = [0.0] * 6
                lstm_position_probabilities = {pos: [] for pos in range(6)}
        else:
            print("⚠️ ข้อมูลใน clean_features_df ไม่เพียงพอสำหรับสร้าง LSTM sequences (ต้องมีอย่างน้อย n_timesteps_lstm งวด).")
            lstm_predicted_digits = ['N/A'] * 6
            lstm_confidence_per_pos = [0.0] * 6
            lstm_position_probabilities = {pos: [] for pos in range(6)}


        lstm_overall_confidence = np.mean([c for c in lstm_confidence_per_pos if c != 0.0]) if any(c != 0.0 for c in lstm_confidence_per_pos) else 0.0

        print(f"\nชุดทำนายที่ 2 (LSTM - ทำนายงวดถัดไป): **{''.join(map(str, lstm_predicted_digits))}**")
        print(f"  ความเชื่อมั่นรวม: {lstm_overall_confidence:.1f}%")
        print("  รายละเอียดแต่ละหลัก:")
        for pos, digit in enumerate(lstm_predicted_digits):
            print(f"    ตำแหน่ง {pos+1}: เลข {digit} (ความเชื่อมั่น: {lstm_confidence_per_pos[pos]:.0f}%)")


        # --- 7. การทำนายเลขท้าย 2 ตัว (สำหรับงวดถัดไป) ---
        print("\n--- 🔢 กำลังทำนายเลขท้าย 2 ตัว (สำหรับงวดถัดไป) ---")

        # Reuse the future_features_df created for 6-digit prediction
        # The 'last2Int' target is independent of 'digits' for prize1

        # Train a separate RandomForest model for last2Int
        rf_model_last2 = RandomForestClassifier(random_state=42, **best_params_rf) # Reuse best RF params

        # Prepare all data for training the last2 model
        # For 'last2Int', we need a target for it.
        # Ensure 'last2Int' is present in clean_features_df for training
        if 'last2Int' not in clean_features_df.columns:
            print("❌ 'last2Int' column not found in clean_features_df. Cannot train last2 prediction model.")
            predicted_last2 = 'N/A'
            last2_confidence = 0.0
            last2_probabilities_top5 = []
        else:
            y_all_last2 = clean_features_df['last2Int']

            # Make sure X_full_scaled (which is X_all_rf in previous code block) is aligned
            # Use the same scaler and feature columns as for 6-digit prediction
            # The 'prepare_ml_data_for_split' function ensures numerical features are scaled correctly
            # We don't need 'digits' in feature_cols_for_prediction when predicting 'last2Int'
            rf_model_last2.fit(X_full_scaled, y_all_last2)

            # Predict last2 for the future features
            proba_last2 = rf_model_last2.predict_proba(X_predict_rf)[0]
            predicted_last2_int = np.argmax(proba_last2)
            last2_confidence = proba_last2[predicted_last2_int] * 100

            predicted_last2 = str(predicted_last2_int).zfill(2)

            # Get top 5 probabilities for last2
            # Ensure that the range is appropriate for your 'last2Int' values (0-99)
            last2_probabilities_top5 = sorted(zip(range(len(proba_last2)), proba_last2), key=lambda x: x[1], reverse=True)[:5]


        print(f"ชุดทำนายเลขท้าย 2 ตัว (RandomForest): **{predicted_last2}**")
        print(f"  ความเชื่อมั่น: {last2_confidence:.1f}%")
        print("  5 อันดับความน่าจะเป็นสูงสุดสำหรับเลขท้าย 2 ตัว:")
        for num, prob in last2_probabilities_top5:
            print(f"    เลข {str(num).zfill(2)}: {prob*100:.1f}%")

        # --- 8. สรุปผลและจัดเก็บข้อมูลสำหรับเซลล์ถัดไป ---
        _ml_analysis_data_cell2 = {
            'rf_best_params': best_params_rf,
            'rf_avg_position_accuracies_backtest': rf_avg_pos_acc,
            'rf_overall_accuracy_backtest': rf_avg_overall_acc,
            'lstm_avg_position_accuracies_backtest': lstm_avg_pos_acc,
            'lstm_overall_accuracy_backtest': lstm_avg_overall_acc,
            'predicted_prize1_rf': ''.join(map(str, rf_predicted_digits)),
            'confidence_prize1_rf': rf_overall_confidence,
            'position_probs_prize1_rf': rf_position_probabilities,
            'predicted_prize1_lstm': ''.join(map(str, [d for d in lstm_predicted_digits if isinstance(d, int)])), # Filter out 'N/A'
            'confidence_prize1_lstm': lstm_overall_confidence,
            'position_probs_prize1_lstm': lstm_position_probabilities,
            'predicted_last2_rf': predicted_last2,
            'confidence_last2_rf': last2_confidence,
            'top5_probs_last2_rf': last2_probabilities_top5,
            'next_draw_date': next_draw_date.strftime('%Y-%m-%d'),
            'feature_columns_used': feature_cols_for_prediction,
            'scaler_for_prediction': scaler_for_prediction
        }

    print("\n--- ✅ ระบบหลัก (Feature Engineering และ Model Logic) เสร็จสมบูรณ์ ---")

⚙️ กำลังสร้าง Sequential Features...
✅ สร้าง Sequential Features สำเร็จ: 560 รายการ

🔬 กำลังดำเนินการ Hyperparameter Tuning สำหรับเลข 6 หลัก (RandomForest)...
  > กำลังทดสอบ n_estimators=50, max_depth=10...
  > กำลังทดสอบ n_estimators=100, max_depth=10...
  > กำลังทดสอบ n_estimators=150, max_depth=10...
  > กำลังทดสอบ n_estimators=200, max_depth=10...


KeyboardInterrupt: 

In [ ]:
# --- เซลล์ที่ 3: ตรวจสอบข้อมูลส่งต่อและ Feature Analysis ---

print("--- เซลล์ที่ 3: ตรวจสอบข้อมูลส่งต่อและ Feature Analysis ---")

# ตรวจสอบว่าตัวแปรที่เก็บผลลัพธ์จากเซลล์ที่ 2 มีอยู่
if '_predictions_cell2' not in globals() or '_analysis_results_cell2' not in globals():
    print("❌ ไม่พบผลลัพธ์การทำนายหรือข้อมูลวิเคราะห์จากเซลล์ที่ 2 โปรดรันเซลล์ที่ 2 ก่อน")
else:
    predictions = _predictions_cell2
    analysis_results = _analysis_results_cell2
    features_data = features # ตัวแปร 'features' ถูกสร้างและเก็บไว้ในขอบเขต global ของเซลล์ 2

    print("\n--- 📊 สรุปข้อมูลวิเคราะห์จากเซลล์ที่ 2 ---")
    print(f"จำนวนข้อมูลที่ประมวลผลแล้ว: {analysis_results['totalRecords']} รายการ")
    print(f"ช่วงข้อมูล: ตั้งแต่ {analysis_results['dataRange']['from']} ถึง {analysis_results['dataRange']['to']}")
    print(f"คุณภาพข้อมูลที่ใช้: {analysis_results['qualityScore']:.2f}%")
    print("การปรับปรุงที่ทำในเซลล์ 1 และ 2:")
    for imp in analysis_results['improvements']:
        print(f"  {imp}")

    print("\n--- 🔍 การตรวจสอบ Features (จาก 'features' ตัวแปร) ---")
    print(f"จำนวน Features ที่สร้าง: {len(features_data)} รายการ")
    print("\nตัวอย่าง Feature ของงวดล่าสุด (เพื่อทำนายงวดถัดไป):")
    if features_data:
        last_feature_set = features_data[-1]
        # แสดงเฉพาะ Features ที่น่าสนใจและเป็นตัวเลข
        print("  Features สำหรับเลข 6 หลัก:")
        for pos in range(6):
            print(f"    pos{pos}_lastValue: {last_feature_set.get(f'pos{pos}_lastValue', 'N/A')}")
            print(f"    pos{pos}_trend: {last_feature_set.get(f'pos{pos}_trend', 'N/A')}")
            print(f"    pos{pos}_stdDev: {last_feature_set.get(f'pos{pos}_stdDev', 'N/A'):.2f}")
        print("  Features ทั่วไป:")
        print(f"    avgSum: {last_feature_set.get('avgSum', 'N/A'):.2f}")
        print(f"    avgEvenCount: {last_feature_set.get('avgEvenCount', 'N/A'):.2f}")
        print("  Features สำหรับเลข 2 ตัวล่าง:")
        print(f"    last2_lastValue: {last_feature_set.get('last2_lastValue', 'N/A')}")
        print(f"    last2_frequency_in_history: {last_feature_set.get('last2_frequency_in_history', 'N/A')}")
        print(f"    last2_gap: {last_feature_set.get('last2_gap', 'N/A')}")
        print("  Features ตามวันที่:")
        print(f"    dayOfWeek: {last_feature_set.get('dayOfWeek', 'N/A')}")
        print(f"    month: {last_feature_set.get('month', 'N/A')}")
        print(f"    dayOfMonth: {last_feature_set.get('dayOfMonth', 'N/A')}")
        print(f"    weekOfYear: {last_feature_set.get('weekOfYear', 'N/A')}")
    else:
        print("  ไม่มี Features ให้แสดง (ข้อมูลอาจไม่เพียงพอ)")

    print("\n--- 📈 ผลลัพธ์การทำนายที่ส่งต่อ (จาก '_predictions_cell2') ---")

    print("\n--- 🔢 เลข 6 หลัก (จาก predict_sequential_6_digits) ---")
    if predictions['sixDigit']['predictions']:
        for i, pred in enumerate(predictions['sixDigit']['predictions']):
            print(f"ชุดทำนายที่ {i+1}: **{pred['sequence']}**")
            print(f"  ความเชื่อมั่น: {pred['confidence']:.1%}")
            print("  รายละเอียดแต่ละหลัก:")
            for detail in pred['breakdown']:
                print(f"    ตำแหน่ง {detail['position']+1}: เลข {detail['digit']} (ความเชื่อมั่น: {detail['confidence']:.0%})")

        print("\n--- การวิเคราะห์แต่ละตำแหน่ง (Position Analysis) ---")
        for pos_analysis in predictions['sixDigit']['positionAnalysis']:
            print(f"ตำแหน่งที่ {pos_analysis['position']+1} (Best Guess: {pos_analysis['bestGuess']}):")
            for cand in pos_analysis['candidates']:
                print(f"  - เลข {cand['digit']} (ความน่าจะเป็น: {cand['probability']:.2%})")
    else:
        print("  ไม่มีการทำนายเลข 6 หลัก (อาจมีข้อมูลไม่เพียงพอ)")

    print("\n--- ⚡ เลข 2 ตัวล่าง (จาก predict_last2_digits) ---")
    if predictions['twoDigit']['predictions']:
        print("🔥 Top Predictions:")
        for i, pred in enumerate(predictions['twoDigit']['predictions']):
            print(f"  {i+1}. **{pred['value']}** - ความเชื่อมั่น: {pred['confidence']:.1%} ({pred['reason']})")

        print("\n📊 Pattern Analysis:")
        analysis_2d = predictions['twoDigit']['analysis']
        print(f"  🎯 คู่ที่ไม่ซ้ำที่พบ: {analysis_2d['totalUniquePairs']} คู่")
        if analysis_2d['mostFrequent']:
            print(f"  🔥 คู่ที่น่าสนใจสุด (จากคะแนน): {analysis_2d['mostFrequent']['pair']} (คะแนน: {analysis_2d['mostFrequent']['score']:.1f})")
        print(f"  ⏰ ช่วงห่างปัจจุบันสูงสุด: {analysis_2d['longestCurrentGap']} งวด")
        print(f"  Pattern Trends (ล่าสุด 5 งวด):")
        print(f"    Sum Mod 10: {' → '.join(map(str, analysis_2d['patterns']['sumMod10Trend']))}")
        print(f"    Even/Odd: {' → '.join(['คี่' if x == 1 else 'คู่' for x in analysis_2d['patterns']['evenOddTrend']])}")
        print(f"  🥶 คู่ที่ 'เย็น' หรือไม่ออกมานาน: {', '.join(analysis_2d['coldPairs'])}")
    else:
        print("  ไม่มีการทำนายเลข 2 ตัวล่าง (อาจมีข้อมูลไม่เพียงพอ)")

    print("\n--- ตรวจสอบข้อมูลเสร็จสิ้น ---")

